In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import os
import sklearn
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization,GlobalAveragePooling2D,HeNormal


KeyboardInterrupt



In [47]:
data_aug_train=ImageDataGenerator(rescale=1.0/255,
                                  rotation_range=30,
                                  shear_range=0.2,
                                  zoom_range=0.15,
                                  horizontal_flip=True,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  channel_shift_range=0.2,
                                  fill_mode='nearest')

In [7]:
data_aug_test=ImageDataGenerator(rescale=1.0/255)

In [49]:
train_gen=data_aug_train.flow_from_directory('train',
                                       target_size=(48,48),
                                       batch_size=16,
                                       class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [50]:
test_gen=data_aug_test.flow_from_directory('2/test',
                                          target_size=(48,48),
                                          batch_size=16,
                                          class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [66]:

model=Sequential([
                  Conv2D(16,kernel_size=(3,3),padding='same',input_shape=(48,48,3),activation='leaky_relu', kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  Conv2D(16,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  
                  Conv2D(32,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  Conv2D(32,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  MaxPooling2D((2,2)),
                  Dropout(rate=0.10),
                  
                  Conv2D(48,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  Conv2D(48,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  BatchNormalization(),
                  MaxPooling2D((2,2)),
                  Dropout(rate=0.10),
    
                  Conv2D(64,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.HeNormal),
                  GlobalAveragePooling2D(),
                  
                  Flatten(),
                  Dense(7,activation='softmax')
                 ])

In [67]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 48, 48, 16)        448       
                                                                 
 batch_normalization_22 (Ba  (None, 48, 48, 16)        64        
 tchNormalization)                                               
                                                                 
 conv2d_35 (Conv2D)          (None, 48, 48, 16)        2320      
                                                                 
 batch_normalization_23 (Ba  (None, 48, 48, 16)        64        
 tchNormalization)                                               
                                                                 
 conv2d_36 (Conv2D)          (None, 48, 48, 32)        4640      
                                                                 
 batch_normalization_24 (Ba  (None, 48, 48, 32)       

In [68]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [69]:
batch_size=16

In [70]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir="./model", histogram_freq=1, write_graph=True, write_images=True,update_freq="epoch")

In [71]:
import time
pre_time = time.time()
model.fit(train_gen,
          steps_per_epoch=28709//batch_size,
          validation_data=test_gen,
          validation_steps=7178//batch_size,
          epochs=32,
          callbacks=[tbCallBack])
print(time.time() - pre_time)

Epoch 1/32
1794/1794 [==============================] - 527s 293ms/step - loss: 1.7701 - accuracy: 0.2702 - val_loss: 1.8245 - val_accuracy: 0.2535
Epoch 2/32
1794/1794 [==============================] - 374s 208ms/step - loss: 1.6356 - accuracy: 0.3476 - val_loss: 1.5831 - val_accuracy: 0.3650
Epoch 3/32
1794/1794 [==============================] - 331s 184ms/step - loss: 1.5241 - accuracy: 0.4156 - val_loss: 1.5723 - val_accuracy: 0.3975
Epoch 4/32
1794/1794 [==============================] - 330s 184ms/step - loss: 1.4480 - accuracy: 0.4439 - val_loss: 1.4242 - val_accuracy: 0.4634
Epoch 5/32
1794/1794 [==============================] - 331s 185ms/step - loss: 1.3944 - accuracy: 0.4691 - val_loss: 1.4546 - val_accuracy: 0.4438
Epoch 6/32
1794/1794 [==============================] - 331s 184ms/step - loss: 1.3479 - accuracy: 0.4868 - val_loss: 1.2653 - val_accuracy: 0.5253
Epoch 7/32
1794/1794 [==============================] - 330s 184ms/step - loss: 1.3190 - accuracy: 0.4976 - val_

In [73]:
path="test/"
filelist=os.listdir(path)
test_sec = 0
count_sec = [0] * 7
true_sec = np.zeros((7,3))
Acc = np.zeros((7,3))
pred_len = 50
for folder in filelist:
    test_sec += 1
    print('sec')
    content_list = []
    content_id = os.listdir(path+folder+'/')
    for i in range(pred_len):
        random_content = random.choice(content_id)
        content_list.append(random_content)
 
    for item in content_list:
        count_sec[test_sec-1] += 1
        img=cv2.imread(path+folder+'/'+item)
        img=cv2.resize(img,(48,48))
        img=np.reshape(img,(1,48,48,3))
        img = img / 255
        pred_prob = model.predict(img)
        for N in range(1,4):
            N_max = np.argsort(pred_prob[0])[-N:]
            if test_sec - 1 in N_max:
                true_sec[test_sec-1][N-1] += 1

sec
1/1 [==============================] - 0s 26ms/step
sec
1/1 [==============================] - 0s 32ms/step
sec
1/1 [==============================] - 0s 52ms/step
sec
1/1 [==============================] - 0s 33ms/step
sec
1/1 [==============================] - 0s 32ms/step
sec
1/1 [==============================] - 0s 39ms/step
sec
1/1 [==============================] - 0s 29ms/step


In [74]:
Acc = true_sec /pred_len
Avg = sum(Acc) / 7
print(Avg)
print(Acc)
print(emotions)

[0.57714286 0.71428571 0.86      ]
[[0.34 0.46 0.78]
 [0.42 0.5  0.68]
 [0.14 0.4  0.7 ]
 [0.94 0.96 1.  ]
 [0.64 0.86 0.96]
 [0.74 0.92 0.98]
 [0.82 0.9  0.92]]
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
